# SPOTIFY
### Group 1:
Zachary Wilkerson, Tara Bode, Hankun Li

### Some things we want to know:
- How many times did a track stay in the daily global top 200 list in 2021?
- How many times did an artist stay in the daily global top 200 list in 2021?

In [1]:
import time
from selenium import webdriver
import pandas as pd
import regex as re

options = webdriver.ChromeOptions();
options.add_argument("disable-geolocation");
options.add_argument("disable-notifications");

driver = webdriver.Chrome(r'/Users/zwilkerson/Desktop/UF/ACG7849 - Web Crawling and Textual Analysis/selenium_files/chromedriver',options=options)

In [2]:
driver.get("https://spotifycharts.com/regional/")

In [3]:
def dict_adder(key,dic):
    try:
        dic[key] += 1;
    except:
        dic[key] = 1;
    return dic

def button_clicking(driver,index):
    button = driver.find_element_by_xpath('//*[@id="content"]/div/div/div/span/div[1]/div/div/div/div[3]')
    button.click()
    new_button = driver.find_element_by_xpath('//*[@id="content"]/div/div/div/span/div[1]/div/div/div/div[3]/ul/li['+str(date_index)+']')
    date = new_button.get_attribute('innerHTML');
    return date,button,new_button

In [4]:
song_dict = {};
artist_dict = {};

date_index = 1;
[date,date_button,newdate_button] = button_clicking(driver,date_index)
date_index = 1;

#while re.search(r'[0-9]{2}/[0-9]{2}/2021',date):
while re.search(r'[0-9]{2}/[0-9]{2}/2021',date):
    newdate_button.click()

    # ranking = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/span/table/tbody/tr/td[2]')
    info = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/span/table/tbody/tr/td[4]')
    # streams = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/span/table/tbody/tr/td[5]')

    # ranking_list = [];
    # streams_list = [];

    for item in range(len(info)):
        song_artist_split = info[item].text.split(' by ')
        dict_adder(song_artist_split[0],song_dict)
        dict_adder(song_artist_split[1],artist_dict)
        # ranking_list.append(ranking[item].text)
        # streams_list.append(streams[item].text)
    
    date_index +=1;
    [date,date_button,newdate_button] = button_clicking(driver,date_index)

date_button.click()
driver.close()
        

## Let's take a look at our data!
Uncomment and run the following lines if you'd like. 

If not, take a look at the printed top artists and songs!  

In [1]:
#artist_dict

In [2]:
#song_dict

In [5]:
# SEE: https://careerkarma.com/blog/python-sort-a-dictionary-by-value/#:~:text=To%20sort%20a%20dictionary%20by%20value%20in%20Python%20you%20can,Dictionaries%20are%20unordered%20data%20structures.
top_artists = sorted(artist_dict.items(), key=lambda x: x[1], reverse=True)[:5]
top_songs = sorted(song_dict.items(), key=lambda x: x[1], reverse=True)[:5]

print('[Top 5 Artists]\n')
for i in top_artists:
    print('{} \t - \t {} occurances'.format(i[0], i[1]))

print('__________________________________________________________________________')
    
print('\n[Top 5 Songs]\n')
for i in top_songs:
    print('{} \t - \t {} occurances'.format(i[0], i[1]))

[Top 5 Artists]

Justin Bieber 	 - 	 749 occurances
Juice WRLD 	 - 	 667 occurances
Billie Eilish 	 - 	 618 occurances
Pop Smoke 	 - 	 604 occurances
Ariana Grande 	 - 	 600 occurances
__________________________________________________________________________

[Top 5 Songs]

Falling 	 - 	 248 occurances
Levitating (feat. DaBaby) 	 - 	 134 occurances
Blinding Lights 	 - 	 134 occurances
The Business 	 - 	 134 occurances
DÁKITI 	 - 	 134 occurances


___
___
___

# FANDANGO
### Group 1:
Zachary Wilkerson, Tara Bode, Hankun Li

### Some things we want to know:
- Of the Movies Now Playing, how highly rated were they by critics and audience members?
- Of the reviews from audience members, how many occurrences of keywords are there of each review and to what extent (i.e. extra positive vs. positive)?

In [3]:
import pandas as pd
import time # needed for time.sleep
from selenium import webdriver
# new driver (opens browser window)
driver = webdriver.Chrome(r'/Users/zwilkerson/Desktop/UF/ACG7849 - Web Crawling and Textual Analysis/selenium_files/chromedriver')
driver.get("https://www.fandango.com/movie-reviews")

In [4]:
el = driver.find_element_by_tag_name("a")
elements = driver.find_elements_by_tag_name("a")
pagesToCrawl = []
for el in elements[0:331]:
    parent = el.find_element_by_xpath('..') 
    if el.get_attribute("class")=="cta mrp__table-movie-reviews-link":      
        pagesToCrawl.append(el.get_attribute("href")  )

In [5]:
# create keyword lists
positive = ['love','fun','good','quality','great']
extra_positive = ['phenomenal','top','heartwarming','favorite','awesome']

negative = [ 'horrible', 'predictable', 'boring', 'bad']
extra_negative = [ 'disappointing', 'worst', 'waste', 'awful','terrible']

In [6]:
movie_list = []
critic_score_list = []
audience_score_list = []
positiveReviews = []
extraPositiveReviews = []
negativeReviews = []
extraNegativeReviews = []

for item in range(len(pagesToCrawl)):
    positiveReviewsCnt = 0
    extraPositiveReviewsCnt = 0
    negativeReviewsCnt = 0
    extraNegativeReviewsCnt = 0
    
    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        critic_score = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        movie_list.append(driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name'));
        critic_score_list.append(critic_score);
        audience_score_list.append(audience_score);
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            for word in positive: 
                positiveReviewsCnt += text.count(word)
            for word in extra_positive: 
                extraPositiveReviewsCnt += text.count(word)
            for word in negative: 
                negativeReviewsCnt += text.count(word) 
            for word in extra_negative: 
                extraNegativeReviewsCnt += text.count(word)
        positiveReviews.append(positiveReviewsCnt)
        extraPositiveReviews.append(extraPositiveReviewsCnt)
        negativeReviews.append(negativeReviewsCnt)
        extraNegativeReviews.append(extraNegativeReviewsCnt)
    except:
        print('No rating score to display for '+pagesToCrawl[item]+' . . . \n')
        movie_list.append(driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name'));
        critic_score_list.append('N/A');
        audience_score_list.append('N/A');
        positiveReviews.append('-')
        extraPositiveReviews.append('-')
        negativeReviews.append('-')
        extraNegativeReviews.append('-')

driver.close()

No rating score to display for https://www.fandango.com/the-birdcage-25th-anniversary-presented-by-tcm-224219/movie-reviews . . . 

No rating score to display for https://www.fandango.com/radhe-your-most-wanted-bhai-2021-224687/movie-reviews . . . 

No rating score to display for https://www.fandango.com/bridesmaids-10th-anniversary-224479/movie-reviews . . . 

No rating score to display for https://www.fandango.com/stand-by-me-35th-anniversary-224478/movie-reviews . . . 

No rating score to display for https://www.fandango.com/the-girl-who-believes-in-miracles-2021-224379/movie-reviews . . . 

No rating score to display for https://www.fandango.com/weathering-with-you-224592/movie-reviews . . . 

No rating score to display for https://www.fandango.com/counter-column-2020-223751/movie-reviews . . . 

No rating score to display for https://www.fandango.com/west-side-story-224693/movie-reviews . . . 



In [7]:
webcrawl = pd.DataFrame(list(zip(movie_list,critic_score_list,audience_score_list,positiveReviews,extraPositiveReviews,negativeReviews,extraNegativeReviews)), columns=['Movie Name','Critic Rating','Audience Rating','Number of Positive Reviews','Number of Extra Positive Reviews','Number of Negative Reviews','Number of Extra Negative Reviews']);
display(webcrawl)

,Movie Name,Critic Rating,Audience Rating,Number of Positive Reviews,Number of Extra Positive Reviews,Number of Negative Reviews,Number of Extra Negative Reviews
0,Spiral: SAW (2021),37%,75%,8,3,1,1
1,Wrath of Man (2021),67%,92%,4,1,0,1
2,Those Who Wish Me Dead (2021),62%,85%,6,0,1,1
3,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,97%,99%,11,0,0,0
4,Raya and the Last Dragon,95%,88%,7,1,0,0
5,Profile (2021),61%,79%,5,0,2,1
6,Mortal Kombat (2021),55%,86%,22,7,14,5
7,Godzilla vs. Kong (2021),75%,91%,4,2,1,1
8,Finding You (2021),63%,89%,5,1,2,0
9,Scoob!,47%,53%,11,2,2,1


In [14]:
#SEE: https://stackoverflow.com/questions/33181846/programmatically-convert-pandas-dataframe-to-markdown-table
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    Markdown(df_formatted.to_clipboard(sep="|", index=False))

pandas_df_to_markdown_table(webcrawl)